# Fase 1 - Pipeline de Extração de Conceitos (PDF -> Markdown -> LLM)

Extração de conceitos técnicos a partir do livro em PDF, utilizando uma abordagem de conversão intermediária para Markdown para preservar a hierarquia dos capítulos.

**Configuração Inicial:** Importa as bibliotecas necessárias e carrega as variáveis de ambiente (chaves de API) para preparar o ambiente de execução.

In [1]:
# Instalação de dependências
!pip install -q pymupdf4llm langchain langchain-openai langchain-anthropic pydantic tqdm python-dotenv ipywidgets

import os
import re
import json
import pymupdf4llm
# Usando tqdm padrão para evitar conflitos de widget
from tqdm import tqdm
from dotenv import load_dotenv
from typing import List, Optional, Dict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser

# Carregar variáveis de ambiente (.env)
load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("AVISO: OPENAI_API_KEY não encontrada no ambiente.")
if not os.getenv("ANTHROPIC_API_KEY"):
    print("AVISO: ANTHROPIC_API_KEY não encontrada no ambiente. A etapa de indução de ontologia falhará.")

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


**Conversão PDF para Markdown:** Converte o arquivo PDF original em texto estruturado Markdown, facilitando a leitura e extração de informações pelo LLM.

In [2]:
def convert_pdf_to_markdown_memory(pdf_path):
    """
    Converte um arquivo PDF para Markdown mantendo o conteúdo em memória.
    Utiliza pymupdf4llm para preservar a estrutura semântica.
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"Arquivo não encontrado: {pdf_path}")
    
    print(f"Convertendo '{pdf_path}' para Markdown...")
    # pymupdf4llm.to_markdown retorna uma string com o conteúdo em Markdown
    md_text = pymupdf4llm.to_markdown(pdf_path)
    print("Conversão concluída.")
    return md_text

**Segmentação de Capítulos:** Divide o texto Markdown completo em capítulos individuais, permitindo um processamento granular e isolado.

In [3]:
def split_markdown_chapters(md_content):
    """
    Segmenta o conteúdo Markdown em capítulos baseando-se no padrão '# **N**'.
    Ignora conteúdo antes do primeiro capítulo numerado.
    """
    # Regex para identificar o início dos capítulos conforme padrão observado: # **<Número>**
    # Captura o número do capítulo
    pattern = r'\n# \*\*(\d+)\*\*'
    
    # Encontrar todas as posições de início de capítulo
    matches = list(re.finditer(pattern, md_content))
    
    chapters = []
    
    if not matches:
        print("Nenhum padrão de capítulo '# **N**' encontrado. Verifique o formato do Markdown.")
        return []

    print(f"Encontrados {len(matches)} marcadores de capítulo.")

    for i, match in enumerate(matches):
        chapter_num = match.group(1)
        start_pos = match.start()
        
        # O fim deste capítulo é o início do próximo, ou o fim do arquivo
        if i + 1 < len(matches):
            end_pos = matches[i+1].start()
        else:
            end_pos = len(md_content)
            
        # Extrair conteúdo
        chapter_content = md_content[start_pos:end_pos].strip()
        
        # Tentar extrair um título da primeira linha ou linhas adjacentes para log/validação
        lines = chapter_content.split('\n')
        header_line = lines[0]
        # Remover o marcador para tentar pegar o resto como título
        title_candidate = re.sub(r'# \*\*\d+\*\*\s*', '', header_line).strip()
        
        # Se o título não estiver na mesma linha, verifica a próxima
        if not title_candidate and len(lines) > 1:
            next_line = lines[1].strip()
            if next_line:
                title_candidate = next_line
        
        if not title_candidate:
            title_candidate = f"Capítulo {chapter_num}"

        # Limita o tamanho do título para exibição
        display_title = (title_candidate[:75] + '..') if len(title_candidate) > 75 else title_candidate
        print(f"  📄 Capítulo {chapter_num}: {display_title}")
        
        chapters.append({
            'chapter_id': chapter_num,
            'title': title_candidate,
            'content': chapter_content
        })
        
    return chapters

**Configuração do Extrator:** Define os esquemas de dados (Schemas) e configura o modelo LLM e o prompt para a extração inicial de conceitos dos textos.

In [4]:
# --- Configuração dos Schemas e LLM de Extração ---

# Schema flexível para a primeira etapa (Extração em Massa)
class Concept(BaseModel):
    nome: str = Field(description="O nome exato do conceito técnico, normalizado (ex: 'ls', '/etc/passwd').")
    tipo: str = Field(description="Categoria técnica específica inferida do texto. Seja detalhista (ex: 'Gerenciador de Pacotes', 'Variável de Ambiente', 'Daemon do Kernel').")
    definicao: str = Field(description="Uma definição técnica focada na utilidade e função dentro do sistema Linux.")
    capitulo_origem: str = Field(description="ID do capítulo de onde foi extraído.")

class ConceptList(BaseModel):
    conceitos: List[Concept]

# Inicializa o modelo extrator (gpt-4o-mini - Rápido e Eficiente)
try:
    llm_extractor = ChatOpenAI(model="gpt-4o-mini", temperature=0)
except Exception as e:
    print(f"Erro ao inicializar ChatOpenAI: {e}")

# Parser
parser = PydanticOutputParser(pydantic_object=ConceptList)

# Prompt Template para Extração Livre
prompt_extractor = ChatPromptTemplate.from_messages([
    ("system", """Você é um Especialista em Knowledge Tracing e Engenharia de Dados.
    Sua tarefa é extrair conceitos técnicos (Knowledge Components) de material didático.
    """),
    ("user", """Analise o texto do Capítulo {chapter_id} ({chapter_title}).

    ### OBJETIVO:
    Extraia TODOS os conceitos técnicos relevantes.
    
    ### REGRAS:
    1. **Canonicalização de Nomes:**
       - Comandos: Apenas o binário (ex: "ls", não "comando ls").
       - Diretórios: Caminho absoluto (ex: "/etc").
    2. **Tipagem Livre:**
       - Não se restrinja a categorias fixas. Identifique o que o objeto É no contexto (ex: "Opção de Boot", "Sistema de Arquivos", "Sigla").
    3. **Definições Ricas:** Explique a função e utilidade técnica.

    {format_instructions}

    ---
    Texto:
    {content}
    """)
])

# Chain de Extração
chain_extractor = prompt_extractor | llm_extractor | parser

print("Configuração de Extração (gpt-4o-mini) concluída.")

Configuração de Extração (gpt-4o-mini) concluída.


**Pipeline de Extração em Massa:** Executa a extração de conceitos em todos os capítulos do livro de forma paralela para otimizar o tempo de processamento.

In [5]:
# --- Pipeline de Extração em Massa (Paralelizado) ---
from concurrent.futures import ThreadPoolExecutor, as_completed

pdf_path = "pdf/linux.pdf"
raw_concepts = [] # Lista para armazenar extração bruta

# Função auxiliar para processar um único capítulo (Isolamento da lógica)
def process_chapter_task(chapter):
    try:
        # Limitar contexto (aprox 40k caracteres para não estourar tokens)
        content_slice = chapter['content'][:40000]
        
        # Invocar gpt-4o-mini
        result = chain_extractor.invoke({
            "chapter_id": chapter['chapter_id'],
            "chapter_title": chapter['title'],
            "content": content_slice,
            "format_instructions": parser.get_format_instructions()
        })
        
        if result and result.conceitos:
            # Retorna lista de dicts
            return [c.model_dump() for c in result.conceitos]
    except Exception as e:
        print(f"\n❌ Erro no capítulo {chapter['chapter_id']}: {str(e)}")
        return []
    return []

# --- Execução Principal ---

if not os.path.exists(pdf_path):
    print(f"❌ Erro: Arquivo {pdf_path} não encontrado.")
else:
    # 1. Converter PDF (se necessário)
    if 'md_content' not in locals():
        md_content = convert_pdf_to_markdown_memory(pdf_path)
    
    # 2. Segmentar Capítulos (se necessário)
    if 'chapters' not in locals() or not chapters:
        chapters = split_markdown_chapters(md_content)
    
    # 3. Loop de Extração Paralela
    if chapters:
        print(f"\n🚀 Iniciando extração PARALELA em massa ({len(chapters)} capítulos)...")
        
        # max_workers=5 é um bom equilíbrio. Se tiver conta Tier 2+ na OpenAI, pode tentar 8 ou 10.
        with ThreadPoolExecutor(max_workers=3) as executor:
            # Submete todas as tarefas
            future_to_chapter = {executor.submit(process_chapter_task, ch): ch for ch in chapters}
            
            # Processa conforme vão ficando prontas (com barra de progresso)
            for future in tqdm(as_completed(future_to_chapter), total=len(chapters), desc="Processando em paralelo"):
                result_concepts = future.result()
                if result_concepts:
                    raw_concepts.extend(result_concepts)

        print(f"\nExtração bruta concluída. Total: {len(raw_concepts)} conceitos.")
    else:
        print("Nenhum capítulo identificado.")

Convertendo 'pdf/linux.pdf' para Markdown...
Conversão concluída.
Encontrados 26 marcadores de capítulo.
  📄 Capítulo 1: Capítulo 1
  📄 Capítulo 2: Capítulo 2
  📄 Capítulo 3: Capítulo 3
  📄 Capítulo 4: Capítulo 4
  📄 Capítulo 5: Capítulo 5
  📄 Capítulo 6: Capítulo 6
  📄 Capítulo 7: Capítulo 7
  📄 Capítulo 8: Capítulo 8
  📄 Capítulo 9: Capítulo 9
  📄 Capítulo 10: Capítulo 10
  📄 Capítulo 11: Capítulo 11
  📄 Capítulo 12: Capítulo 12
  📄 Capítulo 13: Capítulo 13
  📄 Capítulo 14: Capítulo 14
  📄 Capítulo 15: Capítulo 15
  📄 Capítulo 16: Capítulo 16
  📄 Capítulo 17: Capítulo 17
  📄 Capítulo 18: Capítulo 18
  📄 Capítulo 19: Capítulo 19
  📄 Capítulo 20: Capítulo 20
  📄 Capítulo 21: Capítulo 21
  📄 Capítulo 22: Capítulo 22
  📄 Capítulo 23: Capítulo 23
  📄 Capítulo 24: Capítulo 24
  📄 Capítulo 25: Capítulo 25
  📄 Capítulo 26: Capítulo 26

🚀 Iniciando extração PARALELA em massa (26 capítulos)...


Processando em paralelo: 100%|██████████| 26/26 [03:26<00:00,  7.95s/it]


Extração bruta concluída. Total: 360 conceitos.


**Indução de Ontologia:** Utiliza um modelo LLM (GPT-4o) para analisar os tipos de conceitos extraídos e criar dinamicamente uma taxonomia (ontologia) unificada.

In [6]:
# --- Indução de Ontologia (GPT-4o) ---

ontology_map = {}

def induce_ontology(raw_data):
    """
    Usa GPT-4o para criar uma taxonomia canônica a partir dos tipos brutos.
    (Substituído Claude 3.5 Sonnet por GPT-4o para evitar erro 404 e unificar provedor)
    """
    print("Iniciando Indução de Ontologia com GPT-4o...")
    
    # 1. Extrair tipos únicos
    unique_types = list(set([c['tipo'] for c in raw_data]))
    print(f"   Tipos brutos encontrados: {len(unique_types)}")
    
    # Se houver muitos tipos, amostragem ou divisão em batches pode ser necessária,
    # mas para este escopo vamos enviar todos.
    
    try:
        llm_ontologist = ChatOpenAI(
            model="gpt-4o",
            temperature=0
        )
        
        prompt_ontology = ChatPromptTemplate.from_messages([
                    ("system", """Você é um Arquiteto de Ontologias Sênior especializado em Grafos de Conhecimento Educacional (SINKT).
                    
                    Sua tarefa é analisar uma lista de 'tipos de conceitos' extraídos de forma bruta de um livro de Linux e criar uma Taxonomia Canônica (Upper Ontology).
                    
                    DIRETRIZES ESTRITAS:
                    1. Crie entre 6 a 12 categorias mestras (Canônicas) que agrupem os conceitos técnicos.
                    Exemplos sugeridos: 'COMANDO', 'SISTEMA_ARQUIVOS', 'REDE', 'CONCEITO_TEORICO', 'FERRAMENTA', 'HARDWARE', 'PERMISSAO', 'SHELL_SCRIPT'.
                    
                    2. TRATAMENTO DE RUÍDO: Se um tipo bruto não for um conceito técnico ensinável (ex: "Metadado", "Número de Página", "Autor", "Índice", "Dica"), mapeie-o para a categoria especial "NOISE".
                    
                    3. CONSISTÊNCIA: Comandos binários (ls, cd) devem ser 'COMANDO'. Arquivos e pastas (/etc, /bin) devem ser 'SISTEMA_ARQUIVOS'.
                    """),
                    ("user", """Analise a lista de tipos brutos abaixo e mapeie cada um para sua Categoria Canônica.
                    
                    Tipos Brutos Encontrados:
                    {raw_types}
                    
                    Retorne APENAS um JSON válido no formato:
                    {{
                        "map": {{
                            "tipo_bruto_1": "COMANDO",
                            "tipo_bruto_2": "NOISE",
                            "tipo_bruto_3": "SISTEMA_ARQUIVOS"
                        }},
                        "taxonomy": ["COMANDO", "SISTEMA_ARQUIVOS", "REDE", "NOISE", "..."]
                    }}
                    """)
                ])
        chain_ontologist = prompt_ontology | llm_ontologist | JsonOutputParser()
        
        result = chain_ontologist.invoke({"raw_types": json.dumps(unique_types, ensure_ascii=False)})
        
        print("Indução concluída.")
        print(f"   Taxonomia criada: {result.get('taxonomy', [])}")
        return result.get('map', {})
        
    except Exception as e:
        print(f"Falha na Indução de Ontologia: {e}")
        print("   Usando fallback (mapa de identidade)...")
        return {t: t for t in unique_types}

# Executar Indução se houver dados
if raw_concepts:
    ontology_map = induce_ontology(raw_concepts)
else:
    print("Sem dados brutos para induzir ontologia.")

Iniciando Indução de Ontologia com GPT-4o...
   Tipos brutos encontrados: 140
Indução concluída.
   Taxonomia criada: ['COMANDO', 'SISTEMA_ARQUIVOS', 'REDE', 'CONCEITO_TEORICO', 'FERRAMENTA', 'SHELL_SCRIPT', 'NOISE']


**Normalização e Consolidação:** Padroniza os conceitos extraídos utilizando a ontologia criada, removendo duplicatas e unificando definições e metadados.

In [7]:
# --- Normalização e Consolidação Final ---

def normalize_and_consolidate(raw_data, type_map):
    print("Iniciando Normalização e Consolidação...")
    
    # 1. Normalizar Tipos e Filtrar Ruído
    normalized_data = []
    removed_count = 0
    
    for item in raw_data:
        original_type = item['tipo']
        # Usa o mapa, ou mantém original se não encontrado
        canonical_type = type_map.get(original_type, original_type)
        
        # FILTRAGEM DE RUÍDO: Se for mapeado como NOISE, descartamos.
        if canonical_type == "NOISE":
            removed_count += 1
            continue

        # Cria cópia para não mutar original (opcional, mas boa prática)
        new_item = item.copy()
        new_item['tipo'] = canonical_type
        new_item['tipo_original'] = original_type # Guarda histórico
        normalized_data.append(new_item)
    
    print(f"   Itens removidos como 'NOISE': {removed_count}")
        
    # 2. Consolidar (Deduplicação por Nome)
    consolidated = {}
    
    for concept in normalized_data:
        key = concept['nome'].strip().lower()
        
        if key not in consolidated:
            consolidated[key] = concept
            consolidated[key]['_chapter_set'] = {concept['capitulo_origem']}
        else:
            existing = consolidated[key]
            existing['_chapter_set'].add(concept['capitulo_origem'])
            
            # Resolução de conflitos de definição (mantém a mais longa)
            if len(concept['definicao']) > len(existing['definicao']):
                existing['definicao'] = concept['definicao']
                existing['nome'] = concept['nome'] # Preserva casing do vencedor
                
            # Resolução de Tipo:
            # Se já normalizamos, os tipos devem ser iguais.
            # Se houver divergência (ex: mesmo nome mapeado para categorias diferentes por contexto),
            # Priorizamos por frequência ou simplesmente mantemos o que já está.
            # Aqui, como simplificação, mantemos o tipo do conceito com a melhor definição.
            if len(concept['definicao']) > len(existing['definicao']):
                 existing['tipo'] = concept['tipo']

    # Finalizar
    final_list = []
    for item in consolidated.values():
        chapters = sorted(list(item['_chapter_set']), key=lambda x: int(x) if x.isdigit() else 999)
        item['capitulo_origem'] = ", ".join(chapters)
        del item['_chapter_set']
        
        # Opcional: Remover campo auxiliar tipo_original para limpar JSON final
        if 'tipo_original' in item:
            del item['tipo_original']
            
        final_list.append(item)
    
    final_list.sort(key=lambda x: x['nome'].lower())
    
    print(f"Processo Finalizado.")
    print(f"   Conceitos Únicos Consolidados: {len(final_list)}")
    return final_list

if raw_concepts and ontology_map:
    final_concepts = normalize_and_consolidate(raw_concepts, ontology_map)
else:
    final_concepts = []

Iniciando Normalização e Consolidação...
   Itens removidos como 'NOISE': 41
Processo Finalizado.
   Conceitos Únicos Consolidados: 254


**Salvar Artefatos Finais:** Salva os dados processados (conceitos, ontologia e capítulos) em arquivos JSON para persistência e uso nas próximas etapas do pipeline.

In [8]:
# --- Salvar Artefatos Finais ---

output_folder = "output/01_extraction" # 📂 NOVA PASTA
output_ontology_path = f"{output_folder}/ontology_map.json"
output_json_path = f"{output_folder}/concepts_map.json"
output_chapters_path = f"{output_folder}/chapters_content.json"

if final_concepts:
    try:
        # Criar pasta de saída
        os.makedirs(output_folder, exist_ok=True)
        
        # Salvar Conceitos
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(final_concepts, f, indent=4, ensure_ascii=False)
        print(f"💾 Conceitos salvos: {output_json_path}")
        
        # Salvar Mapa Ontológico
        with open(output_ontology_path, 'w', encoding='utf-8') as f:
            json.dump(ontology_map, f, indent=4, ensure_ascii=False)
        print(f"💾 Mapa Ontológico salvo: {output_ontology_path}")

        # Salvar Conteúdo dos Capítulos (IMPORTANTE para a Parte 2)
        if 'chapters' in locals() and chapters:
            with open(output_chapters_path, 'w', encoding='utf-8') as f:
                json.dump(chapters, f, indent=4, ensure_ascii=False)
            print(f"💾 Conteúdo dos capítulos salvo: {output_chapters_path}")
        
        # Preview
        print("\n👁️ Exemplo (Conceito Normalizado):")
        print(json.dumps(final_concepts[:3], indent=2, ensure_ascii=False))
        
    except Exception as e:
        print(f"Erro ao salvar arquivos: {e}")
else:
    print("Nada a salvar.")

💾 Conceitos salvos: output/01_extraction/concepts_map.json
💾 Mapa Ontológico salvo: output/01_extraction/ontology_map.json
💾 Conteúdo dos capítulos salvo: output/01_extraction/chapters_content.json

👁️ Exemplo (Conceito Normalizado):
[
  {
    "nome": "$4LINUX",
    "tipo": "CONCEITO_TEORICO",
    "definicao": "Utilização do operador $ para acessar o valor armazenado na variável 4LINUX, permitindo que o conteúdo da variável seja impresso ou utilizado em comandos.",
    "capitulo_origem": "10"
  },
  {
    "nome": "$PATH",
    "tipo": "CONCEITO_TEORICO",
    "definicao": "A variável de ambiente $PATH contém uma lista de diretórios onde o sistema busca executáveis. Permite que comandos sejam executados sem a necessidade de especificar o caminho absoluto.",
    "capitulo_origem": "12"
  },
  {
    "nome": "--help",
    "tipo": "COMANDO",
    "definicao": "O parâmetro --help é utilizado em comandos externos para fornecer uma consulta rápida sobre os parâmetros e opções disponíveis para aqu